In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.models import vgg16
from torchvision import datasets, transforms

import copy
import time

# 1. Tải model VGG-16 và in ra thông tin mô hình

In [ ]:
model = vgg16(weights=None)  # Sử dụng weights=None thay vì pretrained=True
print("Cấu trúc mô hình VGG-16:")
print(model)

### Hàm để tính số lượng tham số của mô hình

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Số lượng tham số của mô hình ban đầu: {count_parameters(model)}")

# 2. Thực hiện prune một filter và xem sự thay đổi về shape

In [ ]:
def prune_filter(model, layer_index, filter_index):
    conv_layer = model.features[layer_index]
    next_conv_layer = None
    
    # Tìm lớp tích chập tiếp theo
    for layer in model.features[layer_index+1:]:
        if isinstance(layer, nn.Conv2d):
            next_conv_layer = layer
            break
    
    # Tạo một lớp tích chập mới với số filter giảm đi 1
    new_conv = nn.Conv2d(in_channels=conv_layer.in_channels,
                         out_channels=conv_layer.out_channels - 1,
                         kernel_size=conv_layer.kernel_size,
                         stride=conv_layer.stride,
                         padding=conv_layer.padding,
                         dilation=conv_layer.dilation,
                         groups=conv_layer.groups,
                         bias=conv_layer.bias is not None)

    # Sao chép trọng số và bias, ngoại trừ filter được prune
    new_filters = torch.cat((conv_layer.weight.data[:filter_index], conv_layer.weight.data[filter_index+1:]))
    new_conv.weight.data = new_filters

    if conv_layer.bias is not None:
        new_biases = torch.cat((conv_layer.bias.data[:filter_index], conv_layer.bias.data[filter_index+1:]))
        new_conv.bias.data = new_biases

    # Thay thế lớp tích chập cũ bằng lớp mới trong mô hình
    model.features[layer_index] = new_conv

    # Điều chỉnh lớp tích chập tiếp theo nếu có
    if next_conv_layer is not None:
        next_new_conv = nn.Conv2d(in_channels=next_conv_layer.in_channels - 1,
                                  out_channels=next_conv_layer.out_channels,
                                  kernel_size=next_conv_layer.kernel_size,
                                  stride=next_conv_layer.stride,
                                  padding=next_conv_layer.padding,
                                  dilation=next_conv_layer.dilation,
                                  groups=next_conv_layer.groups,
                                  bias=next_conv_layer.bias is not None)
        
        next_new_conv.weight.data = next_conv_layer.weight.data[:, :filter_index, :, :].clone()
        next_new_conv.weight.data = torch.cat([next_new_conv.weight.data, next_conv_layer.weight.data[:, filter_index+1:, :, :]], dim=1)
        
        if next_conv_layer.bias is not None:
            next_new_conv.bias.data = next_conv_layer.bias.data.clone()
        
        # Tìm index của lớp tích chập tiếp theo
        for i, layer in enumerate(model.features[layer_index+1:]):
            if isinstance(layer, nn.Conv2d):
                next_layer_index = layer_index + 1 + i
                break
        
        model.features[next_layer_index] = next_new_conv

    return model

# Prune filter đầu tiên của lớp tích chập đầu tiên
pruned_model = prune_filter(copy.deepcopy(model), 0, 0)


In [ ]:
print("\nShape của lớp tích chập đầu tiên trước khi pruning:")
print(model.features[0].weight.shape)
print("Shape của lớp tích chập đầu tiên sau khi pruning:")
print(pruned_model.features[0].weight.shape)

# Print số lượng tham số sau khi pruning
print(f"\nSố lượng tham số sau khi pruning: {count_parameters(pruned_model)}")

### Chuẩn bị dữ liệu

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Sử dụng một tập dữ liệu nhỏ để demo (1000 ảnh)
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
subset_size = 1000
subset_indices = torch.randperm(len(dataset))[:subset_size]
subset = torch.utils.data.Subset(dataset, subset_indices)
dataloader = torch.utils.data.DataLoader(subset, batch_size=32, shuffle=True)


### Training method

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        model.train()
        running_loss = 0.0
        
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Training Loss: {epoch_loss:.4f}')
    
    return model, epoch_loss

### Train mô hình gốc

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print("\nTraining mô hình gốc...")
start_time = time.time()
_, original_loss = train_model(model, dataloader, criterion, optimizer)
end_time = time.time()
print(f"Thời gian training mô hình gốc: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình gốc: {original_loss:.4f}")

### Train mô hình đã pruning

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pruned_model.parameters(), lr=0.001, momentum=0.9)

print("\nTraining mô hình đã pruning...")
start_time = time.time()
_, pruned_loss = train_model(pruned_model, dataloader, criterion, optimizer)
end_time = time.time()


### Evaluate

In [ ]:
print(f"Thời gian training mô hình đã pruning: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình đã pruning: {pruned_loss:.4f}")

print(f"\nChênh lệch loss: {abs(original_loss - pruned_loss):.4f}")